<a href="https://colab.research.google.com/github/nitish7134/Algorithmic-Toolbox/blob/master/EmotionDetection_Assignment2_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing and importing Packages

In [ ]:
!pip install tensorflow
!pip install -U -q PyDrive
from IPython.display import clear_output
clear_output()


Importing Modules

In [ ]:
#IMPORT ALL MODULES
import io
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import math
import h5py
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

#Utility Functions

## NN using **TensorFlow** and **Keras**

Model Define:


In [ ]:
def defineModel(inputSize,layer_dims=[100,50,25,12,6,3],learningRate=0.001):
  input_layer = tf.keras.Input(shape=(inputSize,))
  prevLayer = input_layer
  for l in range(len(layer_dims)):
    hiddenLayer = Dense(layer_dims[l],activation='relu')(prevLayer)
    prevLayer = hiddenLayer 
  output_layer = Dense(1,activation='sigmoid')(prevLayer)
  model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
  optimiser = tf.keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

  model.compile(optimizer=optimiser, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
def trainModel(model,X,Y,epochs=150,batch_size=1024,verbose=0):
  model.fit(X,Y,epochs=epochs, batch_size=batch_size,shuffle=True, verbose=verbose)
  loss, acc = model.evaluate(X, Y, verbose=verbose)

In [ ]:
def predictNN(model,x,verbose=0):
  predictions = model.predict(x=x,verbose=verbose)
  return predictions

##GA Utility Functions


Filter Features according to chromosome
Parameters : 

> Chromosome (A string of 0 or 1 where 1 means feature at index i is to be considered.)

> Features (Input array containing all the features)



Return :
> Filtered Features.






In [ ]:

def filterFeatures(chromosome,features):
  selected_elements_indices = np.where(chromosome == 1)[0]
  filteredFeatures = features[:, selected_elements_indices]
  return filteredFeatures


Find Fitness of the population
Parameters : 


> Chromosome : Chromosome for which fitness is to be calculated.


> train_X, train_Y, test_X, test_Y : Dataset to check accuracy of NN for fitness function

Return : 
> Fitness of the chromosome







In [ ]:
def fitnessFunction(chromosome, train_X, train_Y, test_X, test_Y):
  trainData = filterFeatures(chromosome,train_X)
  testData = filterFeatures(chromosome,test_X)
  model = defineModel(inputSize = trainData.shape[1])
  trainModel(model,trainData,train_Y)

  result = model.evaluate(testData, test_Y,batch_size=32)
  accuracy = result[1]
  loss = result[0]
  fitness = accuracy-(trainData.shape[1]/(len(chromosome)*10) + loss/1000)
  return fitness



Crossver of two Parents:
Parameters :
> parent1, parent2 : Two Chromosomes

> Method : 0 or 1 



Returns :
> Offspings Of the two Chromosomes




In [ ]:
def GAcrossOver(parent1,parent2,method):
  if method == 0:
    r1 = random.randint(0,parent1.shape[0]-1)
    return np.append(parent1[0:r1],parent2[r1:]),np.append(parent2[0:r1],parent1[r1:])
  elif method == 1:
    r1 = random.randint(0,parent1.shape[0]-1)
    r2 = random.randint(r1+1,parent1.shape[0]-1)
    offspring1 = np.append(np.append(parent1[0:r1],parent2[r1:r2]),parent1[r2:])
    offspring2 = np.append(np.append(parent2[0:r1],parent1[r1:r2]),parent2[r2:]) 
    return offspring1,offspring2

Mutation of a chromosome
Parameters: 
> chromosome : Chromosome to be mutated.

> Method : 0 or 1

Return : 

> Mutated Chromosome







In [ ]:
def GAmutate(chromosome,method):
  if method == 0:
    temp = np.random.randint(2,size=chromosome.shape[0])
    chromosome = np.logical_xor(chromosome,temp)
  elif method ==1:
    r1 = random.randint(0,chromosome.shape[0]-1)
    chromosome = np.append(chromosome[0:r1],np.logical_not(chromosome[r1:]))
  return chromosome

Initial Population:
Parameters: 

> no_of_features 

> populationSize
 
Return:
  Randomly Generated population



In [ ]:
def initialPopulation(no_of_features,populationSize):
  x = np.random.randint(2,size=(populationSize,no_of_features-1))
  y = np.ones(shape=(populationSize,1))
  chromosomePop = np.append(x, y, axis=1)
  return chromosomePop


#Load Data

Auth your Gdrive to read compiled csv file

In [ ]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Update links for the csv file to read the csv files

In [ ]:
## ADD LINKS TO DRIVE CSV FILES
link = "https://drive.google.com/file/d/1GnYwSQvAFE3X4TVYrO_PldUkhT0xB6w0/view"  ##LINK FOR FEAR CSV
link2= "https://drive.google.com/file/d/1f97QxMIiRHVDqiflbM09Y0rbMOY3CI7F/view" ## LINK FOR NOT FEAR CSV
id = link.split("/")[-2] 
id2 = link2.split("/")[-2] 

downloaded = drive.CreateFile({'id':id})  
downloaded2 = drive.CreateFile({'id':id2})  

downloaded.GetContentFile('fear.csv')   
downloaded2.GetContentFile('noFear.csv')

fearData = pd.read_csv('fear.csv') 
noFearData = pd.read_csv('noFear.csv')
fearData = fearData.drop(fearData.columns[0], axis=1)
noFearData = noFearData.drop(noFearData.columns[0], axis=1)

fearArr = fearData.to_numpy()
noFearArr = noFearData.to_numpy()
  
fear = np.ones((len(fearArr), 1))
noFear = np.zeros((len(noFearArr), 1))

mergedArrX = np.concatenate((fearArr,noFearArr),axis=0)
mergedArrY = np.concatenate((fear,noFear),axis=0)
#print(mergedArrX)
#print(mergedArrY.shape)
xTrain, xTest, yTrain, yTest = train_test_split(mergedArrX, mergedArrY, test_size = 0.2, random_state = 0)
xTrain = tf.keras.utils.normalize(xTrain)
yTrain = tf.keras.utils.normalize(yTrain)


ApiRequestError: ignored

#Main

##Feature Selection

Enter Hyperparameters:


In [ ]:
PopulationSize = 10
NumberOfGenerations = 10
MutationRate = 0.05
CrossoverRate = 0.8



Fitness Utility:

In [ ]:
def getFitness(chromosome):
  return fitnessFunction(chromosome,xTrain, yTrain, xTest, yTest)

In [ ]:
import time
startTime = time.time()
chromosomePop = initialPopulation(xTrain.shape[1],PopulationSize)
population = list()
for chromosome in chromosomePop:
  citizen = list()
  citizen.append(getFitness(chromosome))
  citizen.append(chromosome)
  population.append(citizen)
population = sorted(population,key=lambda x : x[0],reverse=True) 
print("Time Taken to create population")
totalTime = time.time()-startTime
print(totalTime)

In [ ]:
sTime = time.time()
print("Generation 0: ")
print("")
print('Best Fitness: {}'.format(population[0][0]))
for gen in range(1,NumberOfGenerations+1):
  newPopulation = list();
  BestCitizen = population[0]
  #Crossover
  for i in range(int(CrossoverRate*PopulationSize)):
    r1 = random.randint(0,PopulationSize-1)
    r2 = random.randint(0,PopulationSize-1)
    offspring1,offspring2 = GAcrossOver(population[r1][1],population[r2][1],random.randint(0,1))
    tempList = list()
    tempList.append(population[r1])
    tempList.append(population[r2])

    citizen1 = list()
    citizen1.append(getFitness(offspring1))
    citizen1.append(offspring1)
    tempList.append(citizen1)

    citizen2 = list()
    citizen2.append(getFitness(offspring2))
    citizen2.append(offspring2)
    tempList.append(citizen2)

    tempList = sorted(tempList,key=lambda x : x[0],reverse=True) 

    newPopulation.append(tempList[0])
    newPopulation.append(tempList[1])

  #Add Remaining
  for i in range(PopulationSize-len(newPopulation)):
    r1 = random.randint(0,PopulationSize-1)
    r2 = random.randint(0,PopulationSize-1)
    newPopulation.append(population[0])
    newPopulation.append(population[1])

  #Mutation
  for i in range(int(MutationRate*PopulationSize)):
    r1 = random.randint(0,PopulationSize-1)
    offspring = GAmutate(newPopulation[r1][1],random.randint(0,1))
    newFitness = getFitness(offspring)
    newPopulation[r1] = [newFitness,offspring]

  population = sorted(newPopulation,key=lambda x : x[0],reverse=True) 
  if(population[0][0]<BestCitizen[0]):
    population[0] = BestCitizen
  print('Generation : {}'.format(gen))
  print('')
  print('Best Fitness: {}'.format(population[0][0]))
  
endTime= time.time()
print("time taken to process all generations:")
timetaken = endTime-sTime
print(timetaken)
BestChromosome = population[0][1]


##Neural Netowork After Feature Selection

In [ ]:
trainData = filterFeatures(BestChromosome,xTrain)
testData = filterFeatures(BestChromosome,xTest)
print("No of Final Features:")
print(trainData.shape[1])
print("No of Models to train on:")
print(trainData.shape[0])

No of Final Features:
311
No of Models to train on:
1509


Enter Hyperparameters:


In [ ]:
layerDims = [450,300,200,130,85,60,40,25,15,10,5,3,1]
epochs=100
batch_size=1024
LearningRate = 0.001

In [ ]:
model = defineModel(trainData.shape[1],layerDims,LearningRate)
trainModel(model,trainData,yTrain,epochs=epochs,batch_size=batch_size,verbose=0)

print("Train Result")
results = model.evaluate(trainData, yTrain, batch_size=batch_size,verbose=0)
print(results)
print('Training Done')  
print("Test Result")
results = model.evaluate(testData, yTest, batch_size=batch_size,verbose=0)
print(results)

Train Result
[0.6084244251251221, 0.9655401110649109]
Training Done
Test Result
[0.6134869456291199, 0.9391534328460693]


In [ ]:
model.summary()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier_knn = KNeighborsClassifier(n_neighbors = 4)
classifier_knn.fit(trainData,yTrain)
#y_pred = classifier_knn.predict(x_test)
trainaccuracy = classifier_knn.score(trainData,yTrain)
print(trainaccuracy)

accuracy = classifier_knn.score(testData,yTest)
print(accuracy)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.9854208084824387
0.8677248677248677


##To Predict on given image:


In [ ]:
#Get Value for input from the Compiled CSV: 

In [ ]:
predictions = predictNN(model,input)
for i in range(len(predictions)):
  print('The person in image with Index  {} is :'.format(i))
  if predictions[i] == 1:
    print("In Fear")
  else:
    print("Not In Fear")
    


ValueError: ignored